# Some experiments with Theorem 1.1

James Yu, 20 December 2021

In [2]:
import numpy as np
from sympy import *

## Is the simple $\epsilon$ weight adjustment valid?

In [2]:
Q = np.identity(2) # I_2

R = np.identity(1) # I_1

B = np.array([
    [0.5], 
    [0.5],
], ndmin = 2) # require B to be a matrix (ndmin = 2), even though one of its dimensions is 1

A = np.array([
    [0.25, 0.25],
    [0.25, 0.25],
], ndmin = 2) # ndmin = 2 isn't necessary here because the matrix has both dimensions > 1, but I keep it for consistency

epsilon = 0.1

A_prime = np.array([
    [0.25 + epsilon, 0.25 - epsilon],
    [0.25 - epsilon, 0.25  + epsilon],
], ndmin = 2)

RHS = np.block([
    [Q, A.T], # A is symmetric but put transpose for consistency with theorem
    [A, -B @ np.linalg.inv(R) @ B.T],
])

LHS = np.block([
    [Q, A_prime.T],
    [A_prime, -B @ np.linalg.inv(R) @ B.T],
])

print(LHS - RHS)

[[ 0.   0.   0.1 -0.1]
 [ 0.   0.  -0.1  0.1]
 [ 0.1 -0.1  0.   0. ]
 [-0.1  0.1  0.   0. ]]


In [3]:
x = np.array([5, -5, -5, 5], ndmin = 2).T
x

array([[ 5],
       [-5],
       [-5],
       [ 5]])

In [4]:
test = LHS - RHS
test @ x

array([[-1.],
       [ 1.],
       [ 1.],
       [-1.]])

In [5]:
x.T @ test @ x

array([[-20.]])

The matrix is not positive semidefinite.

In [6]:
x = np.array([-5, 5, -5, 5], ndmin = 2).T
x.T @ test @ x

array([[20.]])

Nor is it negative semidefinite.

### Conclusion: appears to not be valid in the 2x2 case.

## Theoretical verification:

The above does work standalone as a counterfactual, however.

In [7]:
x1, x2, x3, x4, epsilon = symbols("x1, x2, x3, x4, epsilon")

M = Matrix(([[0, 0, epsilon, -epsilon], [0, 0, -epsilon, epsilon], [epsilon, -epsilon, 0, 0], [-epsilon, epsilon, 0, 0]]))
x = Matrix(([[x1, x2, x3, x4]]))
M

Matrix([
[       0,        0,  epsilon, -epsilon],
[       0,        0, -epsilon,  epsilon],
[ epsilon, -epsilon,        0,        0],
[-epsilon,  epsilon,        0,        0]])

In [8]:
simplify(expand(x * M * x.T))

Matrix([[2*epsilon*(x1*x3 - x1*x4 - x2*x3 + x2*x4)]])

which reduces to $2\epsilon(x_1 - x_2)(x_3 - x_4)$, which is not necessarily non-negative $\forall x$.

## What adjustment to the $A$ matrix would work?

Any working adjustment would need to preserve the weight of the strategic agent through $B$ and thus has to be balanced across columns.

In [11]:
x1, x2, x3, x4, epsilon, delta = symbols("x1, x2, x3, x4, epsilon, delta")

M = Matrix(([[0, 0, epsilon, -epsilon], [0, 0, 0, 0], [epsilon, -epsilon, 0, 0], [0, 0, 0, 0]]))
x = Matrix(([[x1, x2, x3, x4]]))
M

Matrix([
[      0,        0, epsilon, -epsilon],
[      0,        0,       0,        0],
[epsilon, -epsilon,       0,        0],
[      0,        0,       0,        0]])

In [12]:
x * M * x.T

Matrix([[2*epsilon*x1*x3 - epsilon*x1*x4 - epsilon*x2*x3]])

In [13]:
expand(x * M * x.T)

Matrix([[2*epsilon*x1*x3 - epsilon*x1*x4 - epsilon*x2*x3]])

This expression would have to be greater or equal to zero, $\forall x$, for the theorem to apply.

In [14]:
expand(x * M * x.T).subs(x2, 0).subs(x4, 0)

Matrix([[2*epsilon*x1*x3]])

In [10]:
expand(x * M * x.T).subs(x1, 0).subs(x3, 0)

Matrix([[-2*delta*x2*x4]])

These are not necessarily non-negative either with respect to $x$.

## What about adjusting B?

In [2]:
x1, x2, x3, x4, a11, a12, a22, b1, b2, epsilon, delta = symbols("x1, x2, x3, x4, a11, a12, a22, b1, b2, epsilon, delta")

# now this is the A from before
A_tilde = Matrix(([[a11 - epsilon/2, a12 - epsilon/2], [a12 - epsilon/2, a22 - epsilon/2]]))
B_tilde = Matrix(([[b1 + epsilon], [b2 + epsilon]]))

In [3]:
A_tilde

Matrix([
[a11 - epsilon/2, a12 - epsilon/2],
[a12 - epsilon/2, a22 - epsilon/2]])

In [4]:
B_tilde

Matrix([
[b1 + epsilon],
[b2 + epsilon]])

Even reweightings from the naive agents.

In [5]:
simplify(-B_tilde * B_tilde.T) # - B R^-1 B', but suppose R = 1

Matrix([
[            -(b1 + epsilon)**2, -(b1 + epsilon)*(b2 + epsilon)],
[-(b1 + epsilon)*(b2 + epsilon),             -(b2 + epsilon)**2]])

In [1]:
B = Matrix(([[b1], [b2]]))
simplify((-B_tilde * B_tilde.T) - (-B * B.T))

NameError: name 'Matrix' is not defined

In [3]:
M = Matrix(([[0, 0, -epsilon/2, -epsilon/2], 
             [0, 0, -delta/2, -delta/2], 
             [-epsilon/2, -epsilon/2, -(b1**2) - (b1 + epsilon)**2, -b1*b2 - (b1+epsilon)*(b2+delta)], 
             [-delta/2, -delta/2, -b1*b2 - (b1 + epsilon)*(b2 + delta), -(b2**2) - (b2 + delta)**2]]))
M

Matrix([
[         0,          0,                           -epsilon/2,                           -epsilon/2],
[         0,          0,                             -delta/2,                             -delta/2],
[-epsilon/2, -epsilon/2,           -b1**2 - (b1 + epsilon)**2, -b1*b2 - (b1 + epsilon)*(b2 + delta)],
[  -delta/2,   -delta/2, -b1*b2 - (b1 + epsilon)*(b2 + delta),             -b2**2 - (b2 + delta)**2]])

In [4]:
x = Matrix(([[x1, x2, x3, x4]]))
simplify(x * M * x.T)

Matrix([[-x1*(delta*x4 + epsilon*x3)/2 - x2*(delta*x4 + epsilon*x3)/2 - x3*(delta*x2 + epsilon*x1 + 2*x3*(b1**2 + (b1 + epsilon)**2) + 2*x4*(b1*b2 + (b1 + epsilon)*(b2 + delta)))/2 - x4*(delta*x2 + epsilon*x1 + 2*x3*(b1*b2 + (b1 + epsilon)*(b2 + delta)) + 2*x4*(b2**2 + (b2 + delta)**2))/2]])

The term to the right of $x_4$ must be positive since $b_2 > 0$ (this result is irrespective of whether $\delta$ is positive or negative). The whole term then cannot be positive due to the minus sign in front. For example, if $x_4 = 1$, then:

In [5]:
simplify(x * M * x.T).subs(x1, 0).subs(x4, 0).subs(x2, 0)

Matrix([[-x3**2*(b1**2 + (b1 + epsilon)**2)]])

This must be negative. The negative sign is outside the $b_2^2$ term as can be seen by the following:

In [22]:
simplify(x * M * x.T).subs(x1, 0).subs(x3, 0).subs(x2, 0).subs(x4, 1).subs(b2, 0.5)

Matrix([[-(delta + 0.5)**2 - 0.25]])

So this is a counterexample for the $B$ adjustment case, at least for $\delta \neq 0$.